In [1]:
# Recognise Faces using some classification algorithm-like logistic,KNN,etc

# 1.Read a video stream using opencv
# 2.extract faces out of it
# 3. load the training data (numpy array of all the persons)
    # x- values are stored in the numy array
    # y -values we need to assign for each person
# 4. use knn to find the prediction of face (int)
# 5. map the predicted id to name of the user
# 6. Display the predictions on the screen - bounding boc and name

In [1]:
import cv2
import numpy as np
import os

In [2]:
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(train,test,k=5):
    data = []    
    for i in range(train.shape[0]):
        ix = train[i,:-1]
        iy = train[i,-1]
        
        d = distance(test,ix)
        data.append((d,iy))
        
    data = sorted(data,key = lambda x:x[0])
    data = data[:k]
    
    labels = np.array(data)
    labels = labels[:,-1]
    
    output = np.unique(labels,return_counts = True)
    
    index = np.argmax(output[1])
    
    return output[0][index]

In [3]:
cap = cv2.VideoCapture(0)

skip = 0
dataset_path = './data/'

face_data = []
labels = []

class_id = 0 #Labels for the givenn file
names = {} #Mapping btw id - name

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # Create a mapping between class_id and name
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create labels for the class
        target = class_id*np.ones((data_item.shape[0]))
        class_id += 1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels,axis = 0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

#Testing

while True:
    ret,frame = cap.read()
    
    if ret==False:
        continue
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    faces = sorted(faces, key=lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        (x,y,w,h) = face
        
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        #Predicted Label (out)
        out = knn(trainset,face_section.flatten())
        
        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    
    cv2.imshow("Faces",frame)
    
    key_required = cv2.waitKey(1) & 0xFF
    if key_required == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

(34, 30000)
(34, 1)
(34, 30001)
